# tfidfを使って観光地ごとの特徴的な単語を抽出する

ここでは、収集した観光地のレビューから、tf-idfによる観光地ごとの特徴的な単語を算出する

In [5]:
# ライブラリのインポート
import MeCab
import collections
import math
import sqlite3

レビューを読み込む

In [3]:
# 各観光地の名前を配列
places = ['hakataeki', 'uminaka', 'doom', 'kaihinkouen', 'ohori', 'dazaifu']

# 各観光地の口コミデータ1000件を一つにまとめたものを配列する
text_all = []

for p in places:
    with open('data/review/{}.txt'.format(p)) as f:
        s = f.read()
    
    text_all.append(s)

# print(text_all[0])

ある観光地のレビューをまとめたものを一つの文書として、他の観光地の文書との比較でtf-idfを算出する

In [6]:
#---------------------------------macab用----------------------------------------
tagger=MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

#---------------------------------口コミデータから抽出----------------------------------------
#ここで、各観光地の口コミデータ毎の単語の出現回数とtfを値を求める
word_list = []
nouns_list = []

for t in text_all:
  #名詞を格納する配列を用意
  nouns = []

  #名詞を抽出
  for line in tagger.parse(t).splitlines():
    line = line.split(',')
    line[0] = line[0].split('\t')
    if len(line[0]) != 2:
        continue
    if line[0][1] == "名詞":
        nouns.append(line[0][0])
    
  nouns_list.append(nouns)

#文書中の名詞とその出現回数をカウント
  counter = collections.Counter(nouns)

  temp_list = []
  for i in range(len(counter)):
    word = counter.most_common()[i][0]
    count = counter.most_common()[i][1]
    tf = counter.most_common()[i][1] / len(nouns)
    temp = []
    temp = [word,count,tf]

    temp_list.append(temp)

  word_list.append(temp_list)


  #idfを求める関数
def idf(all_doc_count ,doc_count):
  temp = (all_doc_count+1) / (doc_count+1)
  result = math.log(( temp ), 1.5)
  return result

#tf-idfを求める関数
def tf_idf(tf,idf):
  return tf*idf

for i in range(len(text_all)):
  for j in range(len(word_list[i])):
    temp_count = 0  #ある単語を含んでいる文書の数

    for k in range(len(nouns_list)):
      if word_list[i][j][0] in nouns_list[k]:
        temp_count += 1
    
    temp_idf = idf(len(text_all), temp_count) #ある単語のidfを計算
    word_list[i][j].append(temp_idf)

    temp_tf_idf = tf_idf(word_list[i][j][2], temp_idf)  #ある単語のtf-idfを計算
    word_list[i][j].append(temp_tf_idf)  

name_count = 0

for word in word_list:
  list.sort(word, key=lambda x: x[4], reverse = True)
  print("検索した地域:{}".format(places[name_count]))
  name_count += 1
  for i in range(10):
    print("単語：{0}, \t tf-idf値：{1}". format(word[i][0], word[i][4]))
  print("")

検索した地域:hakataeki
単語：お土産, 	 tf-idf値：0.016257937379612255
単語：駅ビル, 	 tf-idf値：0.014434557011355617
単語：新幹線, 	 tf-idf値：0.014247983022216394
単語：阪急, 	 tf-idf値：0.013968926140021565
単語：デパート, 	 tf-idf値：0.009312617426681043
単語：博多口, 	 tf-idf値：0.009312617426681043
単語：玄関口, 	 tf-idf値：0.008381355684012938
単語：買い物, 	 tf-idf値：0.008128968689806127
単語：イルミネーション, 	 tf-idf値：0.007566193934358011
単語：商業施設, 	 tf-idf値：0.006298526551841166

検索した地域:uminaka
単語：イルカ, 	 tf-idf値：0.07252986007585094
単語：水族館, 	 tf-idf値：0.054855917160815997
単語：イルカショー, 	 tf-idf値：0.03784891548785784
単語：ショー, 	 tf-idf値：0.029861887147512176
単語：水槽, 	 tf-idf値：0.02651091437255241
単語：魚, 	 tf-idf値：0.01717932238411881
単語：アシカ, 	 tf-idf値：0.014672707325689385
単語：ペンギン, 	 tf-idf値：0.014051905997288347
単語：マリンワールド, 	 tf-idf値：0.013172089531016607
単語：ラッコ, 	 tf-idf値：0.011340134664478316

検索した地域:doom
単語：野球, 	 tf-idf値：0.08058790232183066
単語：観戦, 	 tf-idf値：0.056411531625281466
単語：試合, 	 tf-idf値：0.04547243128756794
単語：ソフトバンクホークス, 	 tf-idf値：0.025775816665680223
単語：ドーム, 	

観光地ごとの特徴的な単語とそのtf-idfの値をデータベースに保存する

In [8]:

conn = sqlite3.connect('data/tfidf_db.db')
c = conn.cursor()

for i in range(len(places)):  
  # テーブルの作成
  c.execute("create table if not exists {} (word text, tfidf double)" .format(places[i]))

  # データの挿入
  for j in range(len(word_list[i])):
    c.execute("insert into {} values(?,?)" .format(places[i]), [word_list[i][j][0], word_list[i][j][4]])
  

conn.commit()
conn.close()